In [6]:
!apt-get install nvidia-cuda-toolkit

'apt-get' is not recognized as an internal or external command,
operable program or batch file.


In [7]:
!pip install nvidia-cuda-toolkit

ERROR: Could not find a version that satisfies the requirement nvidia-cuda-toolkit (from versions: none)
ERROR: No matching distribution found for nvidia-cuda-toolkit


In [2]:
!nvidia-smi

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
%%writefile vector.cu
#include <iostream>
#include <cstdlib> // Include for rand()
#include <cuda_runtime.h>
using namespace std;

__global__ void add(int* A, int* B, int* C, int size){
  int tid=blockIdx.x * blockDim.x + threadIdx.x;
  if(tid<size){
    C[tid]=A[tid]+B[tid];
  }
}

void initialize(int* vector,int size){
  for(int i=0;i<size;i++){
    vector[i]=rand()% 10;
  }
}
void print(int* vector,int size){
  for(int i=0;i<size;i++){
    cout<<vector[i]<<" ";
  }
  cout<<endl;
}
int main(){
  int N=7;
  int* A,* B,* C;
  int vectorSize=N;
  size_t vectorBytes= vectorSize *sizeof(int);
  A= new int[vectorSize];
  B=new int[vectorSize];
  C=new int[vectorSize];

  initialize(A,vectorSize);
  initialize(B,vectorSize);

  cout<<"vector A:"<<endl;
  print(A,N);
  cout<<"vector B:"<<endl;
  print(B,N);

  int* X,* Y,* Z;
  cudaMalloc(&X, vectorBytes);
  cudaMalloc(&Y,vectorBytes);
  cudaMalloc(&Z, vectorBytes);

  cudaMemcpy(X,A,vectorBytes,cudaMemcpyHostToDevice);
  cudaMemcpy(Y,B,vectorBytes,cudaMemcpyHostToDevice);

  int threadsPerBlock=256;
  int blocksPerGrid=(N+threadsPerBlock-1)/threadsPerBlock;

  add<<<blocksPerGrid,threadsPerBlock>>>(X,Y,Z,N);

  cudaMemcpy(C,Z, vectorBytes, cudaMemcpyDeviceToHost);
  cout<<"Addition: ";
  print(C,N);

  delete[] A;
  delete[] B;
  delete[] C;

  cudaFree(X);
  cudaFree(Y);
  cudaFree(Z);

  return 0;

}

Writing vector.cu


In [4]:
!nvcc -o vector vector.cu

'nvcc' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
!./vector

'.' is not recognized as an internal or external command,
operable program or batch file.
